## Agentics (AG)

AG (shorthand for **Agentics**) is the basic data structure used to **encapsulate data for logical transduction**. You can think of it as a *typed container* that holds a list of Pydantic objects (its `states`) together with their shared schema (the `atype`). Once your data lives inside an `AG`, it becomes a first-class citizen in the Agentics world: it can be mapped, reduced, transformed, and enriched by transducible functions, all while preserving type information and explainability.

In this tutorial, we use `AG` with a simple `Movie` type to show how to:

- define an `atype` (a Pydantic model),
- wrap it in an `AG` instance,
- treat the `AG` as a **typed list** (append, index, iterate),
- and load/save its contents to CSV and JSONL files.

This sets the foundation for later steps, where the same `AG` pattern will power **logical transduction**, Map–Reduce operations, and more complex agentic workflows.

In [ ]:
from agentics import AG
from pydantic import BaseModel
class Movie(BaseModel):
    title: str
    
movies = AG(atype=Movie)
print(movies.states)  ### Initially empty states

movies.states += [Movie(title="The Godfather"), Movie(title="La dolce vita")]  ### add new object
movies.pretty_print()

### Use Agentics as Lists 🧾

An `AG` behaves like a **typed Python list** of Pydantic objects.  
Once you set `atype=YourModel`, the `AG.states` attribute holds a list of instances of that model, and you can:

- append new states (`ag.states.append(...)` or `ag.append(...)`),
- index into them (`ag[0]`, `ag[1]`, …),
- iterate over them (`for state in ag: ...`),
- and use `len(ag)` to get the number of elements.

This means you get all the convenience of ordinary lists, but with the **safety and structure of Pydantic types**, and the ability to plug the same container directly into logical transduction and Map–Reduce workflows.



In [ ]:

for movie in movies:  ### Agentics can be used as lists and allow iteration on states
    print(movie.model_dump_json())

new_movies = AG(
    atype=Movie, states=[Movie(title="Superman")]
)  ### Creating a new agentics to show sum


for state in new_movies:
    movies.append(state)

print(movies.pretty_print())

print(len(movies))

## Input/Output

Agentics states can be initialized loaded and saved to .csv or .jsonl files. AType will be automatically generated from the column names (after they will be normalized as needed for valid pydantic fields), with all attributes set to strings.


In [ ]:
from pathlib import Path

base = Path(".")

# Create a new AG object from the privided csv file
movies = AG.from_csv(base / "data/movies.csv", max_rows=20)

# Note that the type is automatically induced and all arguments are set to srt
print(movies.pretty_print())

# Export the same AG in jsonl format
movies.to_jsonl(base / "data/movies.jsonl")

# Load new agentic from jsonl
movies_copy = AG.from_jsonl(base / "data/movies.jsonl")
for state in movies:
    movies_copy.append(state)

# export the new AG in csv format
movies_copy.to_csv(base / "data/movies_copy.csv")

### Search and Clustering AG States 🔍📎

Agentics enables **semantic search** and **clustering** directly over `AG` states. Once your data is in an `AG`, you can:

- **Search**: embed the states and retrieve the most relevant items for a query (e.g. “crime movies from the 1970s” over a `Movie` AG).
- **Cluster**: group similar states together (e.g. movies by theme, products by use-case, tickets by issue type), using their text fields and schema as input signals.

Underneed, Agentics uses a vector DB to encode all states with sentence embeddings. 

In [ ]:
crime_movies = movies.search("crime movies from the 1970s",k=3)
crime_movies.pretty_print()

In [ ]:
clusters= movies.cluster(n_partitions=3)
for i, cluster in enumerate(clusters):
    print(f"Cluster {i}")
    cluster.pretty_print()  

### Using Agentics with Transducible Functions ⚙️➡️🧠

Agentics and transducible functions are designed to work hand in hand.  
An `AG` gives you a **typed collection of states** (e.g. a list of `Movie`, `Email`, or `Product` objects), while a transducible function defines a **typed transformation** between two Pydantic models (e.g. `Genre << Movie`, `Summary << Email`).

When you combine them:

- each state in an `AG` becomes a structured input to a transducible function,
- the function can be applied over the whole container (via Map / Reduce)

This lets you do things like:

- classify all movies in an `AG`: `Genre << Movie`,
- summarize all tickets: `Summary << Ticket`,
- enrich catalog entries with inferred attributes.

In [ ]:
from agentics.core.transducible_functions import Transduce, transducible
from pydantic import Field, BaseModel
from typing import Optional
class Tweet(BaseModel):
    content: str

class Movie(BaseModel):
    movie_name: Optional[str] = Field(None, description="Movie title.")
    description: Optional[str] = Field(None, description="Short plot summary.")
    year: Optional[int] = Field(None, description="Year of release.")

write_tweet = Tweet << Movie
tweet = await write_tweet([crime_movies.states] * 5)
print(tweet[0])